## Proyecto individual I: steam_games - predicción de precios

In [3]:
#Importacion de librerias
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import skew
from math import sqrt
from numpy import mean, var
import copy 
from sklearn import preprocessing

import json
import ast

In [4]:
# Cargamos el archivo json 
ruta_archivo = 'steam_games.json'  

with open(ruta_archivo, 'r', encoding='latin1') as archivo:
    datos_json = json.load(archivo)

# Convertimos el diccionario a un DataFrame
df = pd.DataFrame([datos_json])

# Mostrar el DataFrame resultante
print(df)
    
#  Este código da un error por el tipo de commillas, procedemos a cambiarlas, además da un error de que cada bloque de datos 
# representa un objeto JSON independiente, pero están separados por líneas. Por defecto, json.load() espera un solo objeto
# JSON completo en el archivo, pero en este caso, tenemos múltiples objetos JSON en diferentes líneas

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [5]:
del df

NameError: name 'df' is not defined

In [6]:
# Este código nos permite leer linea por linea, convertirlo en un diccionario de python y agregarlos a una lista que luego 
# será ingestada en un DataFrame, a su vez la función literal_eval evalúa expresiones literales seguras de Python desde 
# una cadena. Después de iterar por todas las líneas del archivo y convertir los datos en diccionarios de Python, 
# se crea un DataFrame a partir la lista rows, cada diccionario se convierte en una fila del df y cada clave en una columna

rows = []

with open ("steam_games.json") as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))
        
df = pd.DataFrame(rows)

###  1) Proceso de TRANSFORMACIÓN y LIMPIEZA de datos

In [7]:
# Chequeamos que los datos hayan quedado cargados correctamente
df.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


In [8]:
# Paneo general de los datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


##### a. Primero analizaremos la columna precio, ya que a futuro será de vital importancia para el modelo de ML de predicción

In [9]:
df.price.isna().sum()
# Como la cantidad de nan en price no es significativa los eliminamos

1377

In [10]:
df = df.dropna(subset=['price'])

In [11]:
#Columna price sin nan
df.price.isna().sum()

0

In [12]:
# Tenemos que transformar la columna price a tipo float para poder usarla a posterior más facilmente. Para ello empezaré
# a tratar los casos especiales
df.price.unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, 'Free', 10.99, 1.59, 14.99, 1.99, 59.99, 8.99, 6.99,
       7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25, 24.99,
       17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo', 'Play for Free!',
       34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99, 79.99,
       49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99, 149.99, 771.71,
       'Install Now', 21.99, 89.99, 'Play WARMACHINE: Tactics Demo', 0.98,
       139.92, 4.29, 64.99, 'Free Mod', 54.99, 74.99, 'Install Theme',
       0.89, 'Third-party', 0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0,
       5.49, 23.99, 49.0, 20.99, 10.93, 1.39, 'Free HITMAN™ Holiday Pack',
       36.99, 4.49, 2.0, 4.0, 9.0, 234.99, 1.95, 1.5, 199.0, 189.0, 6.66,
       27.99, 10.49, 129.99, 179.0, 26.99, 399.99, 31.99, 399.0, 20.0,
       40.0, 3.33, 199.99, 22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49,
       3.39, 6.0, 19.95, 499.99, 16.06, 4.68, 131.4, 44.98, 2

In [13]:
# Eliminamos las filas de aquellas cadenas de texto que no estamos seguros que su precio sea cero
df.drop(df[(df['price'] == 'Install Theme') | (df['price'] == 'Third-party') | (df['price'] == 'Play Now' )].index,inplace=True)

In [14]:
df.price.unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, 'Free', 10.99, 1.59, 14.99, 1.99, 59.99, 8.99, 6.99,
       7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25, 24.99,
       17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo', 'Play for Free!',
       34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99, 79.99,
       49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99, 149.99, 771.71,
       'Install Now', 21.99, 89.99, 'Play WARMACHINE: Tactics Demo', 0.98,
       139.92, 4.29, 64.99, 'Free Mod', 54.99, 74.99, 0.89, 0.5, 299.99,
       1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99, 10.93, 1.39,
       'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0, 9.0, 234.99,
       1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99, 179.0, 26.99,
       399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99, 22.99, 320.0,
       38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0, 19.95, 499.99, 16.06,
       4.68, 131.4, 44.98, 202.76, 1.0, 2.3, 0.95, 172.24, 249.99, 2.97,

In [15]:
# Definimos el filtro para las filas que deseamos reemplazar
filtro = (df['price'] == 'Free To Play') | (df['price'] == 'Free Demo') | (df['price'] == 'Free to Play') | \
         (df['price'] == 'Play for Free!') | (df['price'] == 'Install Now') | (df['price'] == 'Free Mod') | \
         (df['price'] == 'Free HITMAN™ Holiday Pack') | (df['price'] == 'Play the Demo') | \
         (df['price'] == 'Play WARMACHINE: Tactics Demo') | (df['price'] == 'Free Movie') | \
         (df['price'] == 'Free to Use')  | (df['price'] == 'Free')| \
         (df['price'] == 'Free to Try')    

In [16]:
# Aplicamos el reemplazo usando el filtro
df.loc[filtro, 'price'] = 0

In [17]:
df.price.unique()

array([4.99, 0, 0.99, 2.99, 3.99, 9.99, 18.99, 29.99, 10.99, 1.59, 14.99,
       1.99, 59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99,
       2.49, 15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 34.99,
       74.76, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99, 79.99, 49.99, 5.0,
       44.99, 13.98, 29.96, 119.99, 109.99, 149.99, 771.71, 21.99, 89.99,
       0.98, 139.92, 4.29, 64.99, 54.99, 74.99, 0.89, 0.5, 299.99, 1.29,
       3.0, 15.0, 5.49, 23.99, 49.0, 20.99, 10.93, 1.39, 36.99, 4.49, 2.0,
       4.0, 9.0, 234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49,
       129.99, 179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33,
       199.99, 22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0,
       19.95, 499.99, 16.06, 4.68, 131.4, 44.98, 202.76, 1.0, 2.3, 0.95,
       172.24, 249.99, 2.97, 10.96, 10.0, 30.0, 2.66, 6.48, 19.29, 11.15,
       18.9, 2.89, 99.0, 87.94, 599.0, 8.98, 9.69, 0.49, 9.98, 9.95, 7.0,
       'Starting at $499.00', 'Starting at $449.00'

In [18]:
filtro2 = (df['price'] == 'Starting at $499.00')
filtro3 = (df['price'] == 'Starting at $449.00')

In [19]:
df.loc[filtro2, 'price'] = 499
df.loc[filtro3, 'price'] = 449

In [20]:
df.price.unique()

array([4.99, 0, 0.99, 2.99, 3.99, 9.99, 18.99, 29.99, 10.99, 1.59, 14.99,
       1.99, 59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99,
       2.49, 15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 34.99,
       74.76, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99, 79.99, 49.99, 5.0,
       44.99, 13.98, 29.96, 119.99, 109.99, 149.99, 771.71, 21.99, 89.99,
       0.98, 139.92, 4.29, 64.99, 54.99, 74.99, 0.89, 0.5, 299.99, 1.29,
       3.0, 15.0, 5.49, 23.99, 49.0, 20.99, 10.93, 1.39, 36.99, 4.49, 2.0,
       4.0, 9.0, 234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49,
       129.99, 179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33,
       199.99, 22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0,
       19.95, 499.99, 16.06, 4.68, 131.4, 44.98, 202.76, 1.0, 2.3, 0.95,
       172.24, 249.99, 2.97, 10.96, 10.0, 30.0, 2.66, 6.48, 19.29, 11.15,
       18.9, 2.89, 99.0, 87.94, 599.0, 8.98, 9.69, 0.49, 9.98, 9.95, 7.0,
       499, 449, 12.89, 6.49, 1.87, 42.99, 41.99, 2

In [21]:
# Cambiamos el tipo de dato de price a float
df["price"]=df["price"].astype(float)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30753 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       22982 non-null  object 
 1   genres          27617 non-null  object 
 2   app_name        30752 non-null  object 
 3   title           28820 non-null  object 
 4   url             30753 non-null  object 
 5   release_date    28816 non-null  object 
 6   tags            30622 non-null  object 
 7   discount_price  160 non-null    float64
 8   reviews_url     30751 non-null  object 
 9   specs           30097 non-null  object 
 10  price           30753 non-null  float64
 11  early_access    30753 non-null  bool   
 12  id              30751 non-null  object 
 13  developer       27596 non-null  object 
 14  sentiment       24170 non-null  object 
 15  metascore       2651 non-null   object 
dtypes: bool(1), float64(2), object(13)
memory usage: 3.8+ MB


##### b. Ahora vamos a tratar las columnas sentiment y metascore para dejar provisoriamente los valores faltantes como unknowkn y "0" en metascore, de lo contrario perderemos mucha info valiosa de price que nos servirá para el modelo de prección de precio

In [23]:
print(df.sentiment.isna().sum())
print(df.metascore.isna().sum())

6583
28102


In [24]:
#vemos que no hay score "0" por lo que podemos asignar los nan como "0"
df.metascore.unique()

array([nan, 96, 84, 80, 76, 70, 'NA', 69, 81, 75, 72, 66, 67, 77, 91, 89,
       83, 88, 65, 94, 57, 86, 87, 92, 79, 82, 58, 74, 85, 90, 68, 71, 60,
       73, 59, 64, 61, 54, 53, 78, 51, 44, 63, 38, 56, 49, 52, 62, 93, 48,
       34, 95, 43, 55, 24, 46, 41, 20, 39, 45, 35, 47, 40, 36, 50, 32, 37,
       33, 42, 29, 30], dtype=object)

In [25]:
df.loc[df['metascore'] == 'NA', 'metascore'] = 0

In [26]:
df["metascore"].fillna(0,inplace=True)

In [27]:
df.metascore.unique()

array([ 0, 96, 84, 80, 76, 70, 69, 81, 75, 72, 66, 67, 77, 91, 89, 83, 88,
       65, 94, 57, 86, 87, 92, 79, 82, 58, 74, 85, 90, 68, 71, 60, 73, 59,
       64, 61, 54, 53, 78, 51, 44, 63, 38, 56, 49, 52, 62, 93, 48, 34, 95,
       43, 55, 24, 46, 41, 20, 39, 45, 35, 47, 40, 36, 50, 32, 37, 33, 42,
       29, 30], dtype=int64)

In [28]:
# Cambiamos el tipo de dato a entero
df["metascore"] = df["metascore"].astype(int)

In [29]:
df.sentiment.unique()

array([nan, 'Mostly Positive', 'Mixed', '1 user reviews',
       '3 user reviews', '8 user reviews', 'Very Positive',
       'Overwhelmingly Positive', '6 user reviews', '5 user reviews',
       '2 user reviews', 'Positive', 'Mostly Negative', 'Very Negative',
       '9 user reviews', 'Negative', '4 user reviews', '7 user reviews',
       'Overwhelmingly Negative'], dtype=object)

In [30]:
df["sentiment"].fillna("unknown",inplace=True)

In [31]:
# Definimos el filtro para las filas que deseamos reemplazar (las de pocos reviews)
filtrosentiment = (df['sentiment'] == '1 user reviews') | (df['sentiment'] == '3 user reviews') | \
         (df['sentiment'] == '8 user reviews') | (df['sentiment'] == '6 user reviews') | \
         (df['sentiment'] == '5 user reviews') | (df['sentiment'] == '2 user reviews') | \
         (df['sentiment'] == '9 user reviews') | (df['sentiment'] == '4 user reviews') | \
         (df['sentiment'] == '7 user reviews')   

In [32]:
df.loc[filtrosentiment, 'sentiment'] = "pocos reviews"

In [33]:
df.sentiment.unique()

array(['unknown', 'Mostly Positive', 'Mixed', 'pocos reviews',
       'Very Positive', 'Overwhelmingly Positive', 'Positive',
       'Mostly Negative', 'Very Negative', 'Negative',
       'Overwhelmingly Negative'], dtype=object)

In [34]:
# Cambiamos el tipo de dato de sentiment a categórico
df["sentiment"] = df["sentiment"].astype("category")

In [35]:
# Voy a crear una columna con un ranking de estos sentimientos, que quizás nos sirva para una mejor lectura de datos 
# en el modelo de ML
ranking_map = {
    'unknown': 0,
    'pocos reviews': 0,
    'Very Negative': 1,
    'Negative': 2,
    'Mostly Negative': 3,
    'Mixed': 4,
    'Positive': 5,
    'Mostly Positive': 6,
    'Very Positive': 7,
    'Overwhelmingly Positive': 8
}

df['sentiment_code'] = df['sentiment'].map(ranking_map)
df["sentiment_code"] = df["sentiment_code"].astype('category')


##### c. Tratamos la columna fecha para cambiar el tipo de dato y agregar una columna de año

In [36]:
# Como el numero de nan en release_date no es significativo, los eliminamos
df.release_date.isna().sum()

1937

In [37]:
df = df.dropna(subset=['release_date'])

In [38]:
df.release_date.isna().sum()

0

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28816 entries, 0 to 32133
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   publisher       22978 non-null  object  
 1   genres          27613 non-null  object  
 2   app_name        28816 non-null  object  
 3   title           28816 non-null  object  
 4   url             28816 non-null  object  
 5   release_date    28816 non-null  object  
 6   tags            28687 non-null  object  
 7   discount_price  139 non-null    float64 
 8   reviews_url     28815 non-null  object  
 9   specs           28161 non-null  object  
 10  price           28816 non-null  float64 
 11  early_access    28816 non-null  bool    
 12  id              28815 non-null  object  
 13  developer       27592 non-null  object  
 14  sentiment       28816 non-null  category
 15  metascore       28816 non-null  int32   
 16  sentiment_code  28809 non-null  category
dtypes: bool(1), categ

In [40]:
# aqui cambiar tipo de dato de la fechas a date y crear una columna solo de año con la funcion year()
df["release_date"] = pd.to_datetime(df["release_date"])

ValueError: time data "Jun 2009" doesn't match format "%Y-%m-%d", at position 223. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [41]:
# Al intentar cambiar el formato observé que había fechas en la forma "Jun 2015", por lo que con este código pude detectar
# cuáles eran esas fechas, podría haber cambiado esas filas con loc, pero me di cuenta que todatetime tiene una especificación
# de formato para que pueda reconocer esas fechas

pattern_month_year = r"^[A-Za-z]{3}\s\d{4}$"

# Filtrar las filas que tienen formatos diferentes y guardarlas en una lista
filas_formato_diferente = df[df['release_date'].str.contains(pattern_month_year)]['release_date'].tolist()

print(filas_formato_diferente)

['Jun 2009', 'Oct 2010', 'Oct 2010', 'Oct 2010', 'Feb 2011', 'Sep 2014', 'Apr 2015', 'Apr 2016', 'Jul 2016', 'Jul 2017', 'Jul 2017', 'Apr 2017', 'Jan 2017', 'Nov 2016', 'Nov 2016', 'Oct 2016', 'Jul 2016', 'Jun 2016', 'Aug 2015', 'Jun 2015', 'May 2015', 'Feb 2015', 'Jan 2015', 'Nov 2014', 'Aug 2014', 'Jul 2014', 'May 2014', 'Feb 2013', 'Dec 2012', 'Jul 2010', 'Mar 2010', 'Jan 2010', 'Oct 2009', 'Sep 2009', 'Jun 2009']


In [42]:
""" 
No se hizo pero podría haberse hecho así
df.loc[df["release_date"] == "Jun 2009", 'release_date'] = "2009-06-01"
df.loc[df["release_date"] == "Oct 2010", 'release_date'] = "2010-10-01"
df.loc[df["release_date"] == "Feb 2011", 'release_date'] = "2011-02-01"
df.loc[df["release_date"] == "Apr 2015", 'release_date'] = "2015-04-01"
df.loc[df["release_date"] == "Apr 2016", 'release_date'] = "2016-04-01"
df.loc[df["release_date"] == "Jul 2016", 'release_date'] = "2016-07-01"
df.loc[df["release_date"] == "Jul 2017", 'release_date'] = "2017-07-01" 

""" 

' \nNo se hizo pero podría haberse hecho así\ndf.loc[df["release_date"] == "Jun 2009", \'release_date\'] = "2009-06-01"\ndf.loc[df["release_date"] == "Oct 2010", \'release_date\'] = "2010-10-01"\ndf.loc[df["release_date"] == "Feb 2011", \'release_date\'] = "2011-02-01"\ndf.loc[df["release_date"] == "Apr 2015", \'release_date\'] = "2015-04-01"\ndf.loc[df["release_date"] == "Apr 2016", \'release_date\'] = "2016-04-01"\ndf.loc[df["release_date"] == "Jul 2016", \'release_date\'] = "2016-07-01"\ndf.loc[df["release_date"] == "Jul 2017", \'release_date\'] = "2017-07-01" \n\n'

In [43]:
def contains_word(text):
    return any(word.isalpha() for word in text.split())

# Filtrar las filas que contienen palabras y guardarlas en una lista
filas_con_palabras = df[df['release_date'].apply(contains_word)]['release_date'].tolist()

print(filas_con_palabras)
print(len(filas_con_palabras))

#Vemos en el resultado que las dos diferentes son SOON y coming soon, podemos filtrar esos dos valores para ver que
# tantos datos son, si no es significativo podemos aplicar error=coerse en la función todatetime así esas vellas queden
# como nan y luego las eliminamos

['Jun 2009', 'Oct 2010', 'Oct 2010', 'Oct 2010', 'Feb 2011', 'Sep 2014', 'Apr 2015', 'Apr 2016', 'Jul 2016', 'Jul 2017', 'SOON', 'Jul 2017', 'Apr 2017', 'Jan 2017', 'Nov 2016', 'Nov 2016', 'Oct 2016', 'Jul 2016', 'Jun 2016', 'Aug 2015', 'Jun 2015', 'May 2015', 'Feb 2015', 'Jan 2015', 'Nov 2014', 'Aug 2014', 'Jul 2014', 'May 2014', 'Feb 2013', 'Dec 2012', 'Jul 2010', 'Mar 2010', 'Jan 2010', 'Oct 2009', 'Sep 2009', 'Jun 2009', 'coming soon']
37


In [44]:
soon = (df['release_date'] == 'SOON')| (df['release_date'] == 'coming soon')
cantidad_soon = soon.sum()
print(cantidad_soon)
# Por lo tanto decido eliminarlos
df.drop(df[soon].index, inplace=True)

2


In [45]:
# Ahora ya estamos en condiciones de cambiar el tipo de dato a todatetime, dejaremos como error las que están mal escritas
# ya que no suponen un número significativo (solo 35)
df["release_date"] = pd.to_datetime(df["release_date"], format='%Y-%m-%d', errors="coerce")

In [46]:
df.release_date.isna().sum()

37

In [47]:
df = df.dropna(subset=['release_date'])

In [48]:
df.release_date.isna().sum()

0

In [49]:
# Por último, creamos una columna de año para poder hacer más facilmente las funciones de la API
df['año'] = df['release_date'].dt.year

##### d. Miramos si hay duplicados en las columnas title y id para no repetir peliculas en el modelo 

In [50]:
# Supongamos que ya tienes el DataFrame df y quieres verificar duplicados en la columna 'mi_columna'
columna_duplicados = df['id'].duplicated()
columna_duplicados2 = df['title'].duplicated()
# Para obtener los registros duplicados, puedes filtrar el DataFrame
duplicados = df[columna_duplicados]
duplicados2 = df[columna_duplicados2]
# Para ver cuántos valores duplicados hay en total en la columna:
total_duplicados = columna_duplicados.sum()
total_duplicados2 = columna_duplicados2.sum()
# Para verificar si hay algún valor duplicado en la columna:
hay_duplicados = any(columna_duplicados)
hay_duplicados2 = any(columna_duplicados2)
# Imprimir resultados
print("hay", total_duplicados, "duplicados en la columna id y" , total_duplicados2 , "duplicados en la columna title" )
print("¿Hay duplicados en id?", hay_duplicados, "¿Hay duplicados en title?", hay_duplicados2)


hay 1 duplicados en la columna id y 27 duplicados en la columna title
¿Hay duplicados en id? True ¿Hay duplicados en title? True


In [51]:
df.drop_duplicates(subset=['title'], inplace=True)
df.drop_duplicates(subset=['id'], inplace=True)

##### e. Eliminamos vacios del resto de las columnas (que no sean significativos en número) y columnas que no vayamos a utilizar dados los requerimientos del proyecto

In [52]:
df.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore,sentiment_code,año
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,unknown,0,0.0,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,Secret Level SRL,Mostly Positive,0,6.0,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,Poolians.com,Mostly Positive,0,6.0,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,unknown,0,0.0,2017
5,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",NaN,http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd,Mixed,0,4.0,2018


In [53]:
no_son_iguales = df['app_name'] != df['title']

# Contar cuántas filas tienen valores diferentes en ambas columnas:
num_filas_no_iguales = no_son_iguales.sum()

print(f"Hay {num_filas_no_iguales} filas en las que las columnas no son iguales.")

# Como el número de filas en las que ambas columnas no son iguales es muy pequeño (aprox 1%) vamos a eliminar una de ellas

Hay 544 filas en las que las columnas no son iguales.


In [54]:
# Podemos observar que probablemente app_name y title tienen la misma información, vamos a compararla para saber que
# a ciencia cierta si esto es verdad

no_son_iguales = df['publisher'] != df['developer']

# Contar cuántas filas tienen valores diferentes en ambas columnas:
num_filas_no_iguales = no_son_iguales.sum()

print(f"Hay {num_filas_no_iguales} filas en las que las columnas no son iguales.")

# Aqui no podemos eliminar, ya que hay muchas diferencias entre columnas y quizás podamos correlacionar al developer o 
# publisher con el precio, a futuro, en el modelo de ML

Hay 16992 filas en las que las columnas no son iguales.


In [55]:
# Eliminamos las columnas tags (nos quedamos con specs ya que la usaremos en una de nuestras funciones de la api), 
# discount_price (tiene muy pocos datos y solo usaremos price para el modelo de ML), reviews_url (no es necesaria),
# url (innecesaria), app_name (redundante, title tiene la misma info)
columnas_a_eliminar = ['app_name', 'tags',"url","reviews_url","discount_price"]
df_sin_columnas = df.drop(columnas_a_eliminar, axis=1,inplace=True)

In [56]:
df.head()


,publisher,genres,title,release_date,specs,price,early_access,id,developer,sentiment,metascore,sentiment_code,año
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,[Single-player],4.99,False,761140,Kotoshiro,unknown,0,0.0,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,Secret Level SRL,Mostly Positive,0,6.0,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,Poolians.com,Mostly Positive,0,6.0,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,[Single-player],0.99,False,767400,彼岸领域,unknown,0,0.0,2017
5,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd,Mixed,0,4.0,2018


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28750 entries, 0 to 32133
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   publisher       22921 non-null  object        
 1   genres          27550 non-null  object        
 2   title           28750 non-null  object        
 3   release_date    28750 non-null  datetime64[ns]
 4   specs           28096 non-null  object        
 5   price           28750 non-null  float64       
 6   early_access    28750 non-null  bool          
 7   id              28750 non-null  object        
 8   developer       27529 non-null  object        
 9   sentiment       28750 non-null  category      
 10  metascore       28750 non-null  int32         
 11  sentiment_code  28743 non-null  category      
 12  año             28750 non-null  int32         
dtypes: bool(1), category(2), datetime64[ns](1), float64(1), int32(2), object(6)
memory usage: 2.3+ MB


In [58]:
# Imputamos como unknown a publisher, developer, specs y genres por el momento. Luego veremos si no hacen falta las columnas
# publisher y developer las eliminamos. Y si no influye mucho, eliminamos los nan de specs y genres
df["genres"].fillna("unknown",inplace=True)
df["specs"].fillna("unknown",inplace=True)
df["developer"].fillna("unknown",inplace=True)
df["publisher"].fillna("unknown",inplace=True)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28750 entries, 0 to 32133
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   publisher       28750 non-null  object        
 1   genres          28750 non-null  object        
 2   title           28750 non-null  object        
 3   release_date    28750 non-null  datetime64[ns]
 4   specs           28750 non-null  object        
 5   price           28750 non-null  float64       
 6   early_access    28750 non-null  bool          
 7   id              28750 non-null  object        
 8   developer       28750 non-null  object        
 9   sentiment       28750 non-null  category      
 10  metascore       28750 non-null  int32         
 11  sentiment_code  28743 non-null  category      
 12  año             28750 non-null  int32         
dtypes: bool(1), category(2), datetime64[ns](1), float64(1), int32(2), object(6)
memory usage: 2.3+ MB


In [60]:
df.head()

,publisher,genres,title,release_date,specs,price,early_access,id,developer,sentiment,metascore,sentiment_code,año
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,[Single-player],4.99,False,761140,Kotoshiro,unknown,0,0.0,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,Secret Level SRL,Mostly Positive,0,6.0,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,Poolians.com,Mostly Positive,0,6.0,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,[Single-player],0.99,False,767400,彼岸领域,unknown,0,0.0,2017
5,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd,Mixed,0,4.0,2018


### 2) Creación de funciones a usar en la API

a. Función que devuelve el top 5 de los géneros ingresando el año que se quiere revisar

In [75]:
def genres(year):
    # Filtrar el DataFrame para obtener solo los datos del año específico
    df_year = df[df["año"] == int(year)]
    
    # Crear un diccionario para contar la cantidad de veces que aparece cada género
    genre_counts = {}
    for genres_list in df_year['genres']:
        for genre in genres_list:
            genre_counts[genre] = genre_counts.get(genre, 0) + 1
    
    # Ordenar el diccionario por los valores (cantidad de ventas) en orden descendente
    sorted_genres = sorted(genre_counts.items(), key=lambda x: x[1], reverse=True)
    
    # Obtener los 5 géneros más vendidos
    top_genres = [genre for genre, _ in sorted_genres[:5]]
    
     # Crear un diccionario para el resultado final
    result_dict = {}
    for genre in top_genres:
        # Obtener la cantidad de juegos de cada género en el año específico
        count = genre_counts.get(genre, 0)
        result_dict[genre] = count
    
    return result_dict

In [76]:
genres(2010)

{'Action': 147, 'n': 126, 'Indie': 108, 'Strategy': 101, 'Adventure': 84}

b. Función que devuelve una lista de los juegos lanzados en el año que se quiera revisar

In [65]:
def juegos(year):
    # Filtrar el DataFrame para obtener solo los datos del año específico
    df_year = df[df["año"] == int(year)]

    # Obtener la lista de juegos del año específico
    juegos_list = df_year['title'].tolist()

    juegos_dict = {year: juegos_list}

    return juegos_dict

In [66]:
juegos(2015)

{2015: ['Assault Android Cactus',
  'The Tower',
  'Dimmdrive :: Gaming Ramdrive @ 10,000+ MB/s',
  'Fake Colours',
  'WolfWars',
  'Dr.Green',
  'Vintage Year',
  'The Race for the White House',
  'X-Plane 10 AddOn - Carenado - C185F Skywagon',
  'Rocksmith® 2014 – Linkin Park - “What I’ve Done”',
  'Rocksmith® 2014 – Linkin Park Song Pack',
  'Antisquad: Tasks near the coast of Somalia. Tactics DLC',
  'Castle',
  'Antisquad: Tasks in North Korea. Tactics DLC',
  'ATLAS レジェンドパック',
  'Ostrich Island',
  'Adventures of Robinson Crusoe',
  'Mechs &amp; Mercs: Black Talons',
  'Urja',
  'Teddy Floppy Ear - Mountain Adventure',
  'Su-27: The Ultimate Argument Campaign',
  'Squirreltopia',
  'Lara Croft and the Temple of Osiris - Icy Death Pack',
  'Lara Croft and the Temple of Osiris - Legend Pack',
  'Lara Croft and the Temple of Osiris - Deus Ex Pack',
  'Disorder',
  'Lara Croft and the Temple of Osiris - Hitman Pack',
  'Avernum 2: Crystal Souls',
  'Gunslugs 2',
  "Train Simulator: C

c. Función que devuelve el top 5 de los specs ingresando el año que se quiere revisar

In [80]:
def specs(year):
    # Filtrar el DataFrame para obtener solo los datos del año específico
    df_year = df[df["año"] == int(year)]
    
    # Crear un diccionario para contar la cantidad de veces que aparece cada género
    spec_counts = {}
    for specs_list in df_year['specs']:
        for spec in specs_list:
            spec_counts[spec] = spec_counts.get(spec, 0) + 1
    
    # Ordenar el diccionario por los valores (cantidad de ventas) en orden descendente
    sorted_specs = sorted(spec_counts.items(), key=lambda x: x[1], reverse=True)
    
    # Obtener los 5 géneros más vendidos
    top_specs = [spec for spec, _ in sorted_specs[:5]]
    
    return top_specs

In [81]:
specs(2015)

['Single-player',
 'Steam Achievements',
 'Steam Trading Cards',
 'Downloadable Content',
 'Steam Cloud']

d. Función que devuelve la cantidad de juegos que fueron lanzados en un año determinado con early acces

In [69]:
def earlyacces(year):
    # Filtrar el DataFrame para obtener solo los datos del año específico y que tengan Early Access
    df_year_early_access = df[(df["año"] == int(year)) & (df["early_access"] == True)]
    
    # Obtener la cantidad de juegos con Early Access
    cantidad_juegos_early_access = len(df_year_early_access)
    
    return cantidad_juegos_early_access

In [70]:
earlyacces(2015)

218

e. Función que devuelve un diccionario con los sentimientos y su cantidad en un año específico que le pedimos

In [71]:
def sentiment(year):
    # Filtrar el DataFrame para obtener solo los datos del año específico
    df_year = df[df["año"] == int(year)]
    
    # Contar la cantidad de registros para cada categoría de sentimiento y lo convertimos en un diccionario
    sentiment_counts = df_year['sentiment'].value_counts().to_dict()
    
    return sentiment_counts


In [72]:
sentiment(2015)

{'pocos reviews': 1256,
 'Mixed': 792,
 'unknown': 736,
 'Very Positive': 626,
 'Positive': 481,
 'Mostly Positive': 471,
 'Mostly Negative': 152,
 'Overwhelmingly Positive': 52,
 'Negative': 31,
 'Very Negative': 2,
 'Overwhelmingly Negative': 1}

f. Función que devuelve los 5 juegos con mayor metascore en el año específico que ingresamos

In [73]:
def metascore(year):
    # Filtrar el DataFrame para obtener solo los datos del año específico
    df_year = df[df["año"] == int(year)]
    
    # Ordenar el DataFrame por el metascore en orden descendente
    df_sorted = df_year.sort_values(by='metascore', ascending=False)
    
    # Obtener los 5 juegos con mayor metascore
    top_5_games = df_sorted.head(5)["title"].to_list()
    
    return top_5_games

In [74]:
metascore(2015)

['Grand Theft Auto V',
 'Divinity: Original Sin - Enhanced Edition',
 'Undertale',
 'METAL GEAR SOLID V: THE PHANTOM PAIN',
 'Pillars of Eternity']

### 3) Guardamos el proceso en un nuevo csv limpio

In [83]:
nombre_archivo = 'steam_games_limpio.csv'
df.to_csv(nombre_archivo, index=False)